In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

## The Oscar Winners 
The Academy Awards, 1927 - 2020

In [2]:
#Import CSV File
csv_file = "Data/the_oscar_award.csv"
Oscar_df = pd.read_csv(csv_file)
Oscar_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [3]:
Oscar_group=Oscar_df.groupby('film')
OscarCount = Oscar_group.count()

In [4]:
Oscar_reset=OscarCount.reset_index()
Oscar_reset.head()

,film,year_film,year_ceremony,ceremony,category,name,winner
0,"$1,000 a Minute",1,1,1,1,1,1
1,'38',1,1,1,1,1,1
2,'Crocodile' Dundee,1,1,1,1,1,1
3,'Round Midnight,2,2,2,2,2,2
4,(A) Torzija [(A) Torsion],1,1,1,1,1,1


In [5]:
OscarID = Oscar_reset[['film','year_film']]
OscarID = OscarID.rename(columns={"year_film":"oscar_nominations"})
OscarID

,film,oscar_nominations
0,"$1,000 a Minute",1
1,'38',1
2,'Crocodile' Dundee,1
3,'Round Midnight,2
4,(A) Torzija [(A) Torsion],1
...,...,...
4828,the end,1
4829,tom thumb,1
4830,À Nous la Liberté,1
4831,Ådalen '31,1


In [6]:
#Clean Dataframe
Oscar1_df = Oscar_df[['year_film','category','name','film','winner']]
Oscar1_df = Oscar1_df.rename(columns={"category":"award",
                                          "year_film":"release_year",
                                          "name":"name",
                                          "film":"title",
                                          "winner":"winner"})
Oscar1_df = Oscar1_df.set_index("award")
Oscar1_df.head()

,release_year,name,title,winner
award,,,,
ACTOR,1927,Richard Barthelmess,The Noose,False
ACTOR,1927,Emil Jannings,The Last Command,True
ACTRESS,1927,Louise Dresser,A Ship Comes In,False
ACTRESS,1927,Janet Gaynor,7th Heaven,True
ACTRESS,1927,Gloria Swanson,Sadie Thompson,False


In [7]:
OscarNom_df = Oscar1_df.loc[["BEST PICTURE","DIRECTING","ACTOR","ACTRESS"]]
OscarNom_df

,release_year,name,title,winner
award,,,,
BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...
ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
ACTRESS,1975,Ann-Margret,Tommy,False
ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True


In [8]:
OscarWin_df = OscarNom_df.loc[OscarNom_df["winner"] == True, :]
OscarWin_df.head(1)

,release_year,name,title,winner
award,,,,
BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True


In [9]:
#Reset Index on OscarNom_df
OscarNom_df = OscarNom_df.reset_index()
OscarNom_df = OscarNom_df[['award','release_year','name','title','winner']]
OscarNom_df

,award,release_year,name,title,winner
0,BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
1,BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
2,BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
3,BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
4,BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...,...
1245,ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
1246,ACTRESS,1975,Ann-Margret,Tommy,False
1247,ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True
1248,ACTRESS,1975,Glenda Jackson,Hedda,False


In [10]:
#Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
#Verify
engine.table_names()

['oscarindex', 'oscars', 'movie_meta', 'credits', 'ratings']

In [12]:
OscarID.to_sql(name='oscarindex', con=engine, if_exists='append', index=False)

In [13]:
#Confirm successful upload
pd.read_sql_query('select * from oscarindex', con=engine)

,id,film,oscar_nominations
0,1,"$1,000 a Minute",1
1,2,'38',1
2,3,'Crocodile' Dundee,1
3,4,'Round Midnight,2
4,5,(A) Torzija [(A) Torsion],1
...,...,...,...
4828,4829,the end,1
4829,4830,tom thumb,1
4830,4831,À Nous la Liberté,1
4831,4832,Ådalen '31,1


In [14]:
OscarNom_df.to_sql(name='oscars', con=engine, if_exists='append', index=False)

In [15]:
#Confirm successful upload
pd.read_sql_query('select * from oscars', con=engine)

,award,release_year,name,title,winner
0,BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
1,BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
2,BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
3,BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
4,BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...,...
1245,ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
1246,ACTRESS,1975,Ann-Margret,Tommy,False
1247,ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True
1248,ACTRESS,1975,Glenda Jackson,Hedda,False


## The Movies Dataset
Metadata on over 45,000 movies. 26 million ratings from over 270,000 users.

In [16]:
# Import data

credits_df = pd.read_csv("Data/credits.csv", low_memory=False)
metadata_df = pd.read_csv("Data/movies_metadata.csv", low_memory=False)
ratings_df = pd.read_csv("Data/ratings.csv")

In [17]:
# What data do we have in the movie metadata file?

metadata_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [18]:
# Remove unwanted columns from movie metadata & clean rows (drop duplicates and errors)

metadata_df = metadata_df[["id","title","release_date","runtime","genres","adult","budget",
                           "revenue","production_companies","production_countries", "imdb_id"]]
metadata_df.drop_duplicates("id",inplace=True)
metadata_df = metadata_df.loc[(metadata_df["id"] != "1997-08-20") & (metadata_df["id"] != "2012-09-29") & (metadata_df["id"] != "2014-01-01")]
metadata_df = metadata_df.loc[(metadata_df["budget"] != 0) & (metadata_df["revenue"] != 0) ]
metadata_df

,id,title,release_date,runtime,genres,adult,budget,revenue,production_companies,production_countries,imdb_id
0,862,Toy Story,1995-10-30,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",False,30000000,373554033.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114709
1,8844,Jumanji,1995-12-15,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",False,65000000,262797249.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113497
3,31357,Waiting to Exhale,1995-12-22,127.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,16000000,81452156.0,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114885
4,11862,Father of the Bride Part II,1995-02-10,106.0,"[{'id': 35, 'name': 'Comedy'}]",False,0,76578911.0,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113041
5,949,Heat,1995-12-15,170.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",False,60000000,187436818.0,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113277
...,...,...,...,...,...,...,...,...,...,...,...
45399,280422,All at Once,2014-06-05,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",False,750000,3.0,"[{'name': 'Кинокомпания «Lunapark»', 'id': 420...","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",tt3805180
45401,240789,The Miracle,2009-10-09,110.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",False,0,50656.0,"[{'name': 'Central Partnership', 'id': 9342}, ...","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",tt1197626
45409,62757,Savages,2006-11-23,100.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,800000,1328612.0,[],[],tt0933361
45412,63281,Pro Lyuboff,2010-09-30,107.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",False,2000000,1268793.0,"[{'name': 'Profit', 'id': 21163}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",tt1718881


In [19]:
# Rename columns in credits_df to match SQL tables

credits_df = credits_df.rename(columns={"cast":"movie_cast","crew":"movie_crew"})
credits_df.drop_duplicates("id",inplace=True)
credits_df.dropna(how="any")
credits_df

,movie_cast,movie_crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [20]:
# Group ratings_df by movieId, taking average of all ratings in dataset for each movie

ratings_df = ratings_df[["movieId","rating"]]
grouped_ratings = ratings_df.groupby(["movieId"]).mean()
grouped_ratings = pd.DataFrame(grouped_ratings)
grouped_ratings.dropna()
grouped_ratings

,rating
movieId,
1,3.888157
2,3.236953
3,3.175550
4,2.875713
5,3.079565
...,...
176267,4.000000
176269,3.500000
176271,5.000000


In [21]:
# Connect to postgresql

rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

engine.table_names()

['oscarindex', 'oscars', 'movie_meta', 'credits', 'ratings']

In [22]:
# Export movie metadata

metadata_df.to_sql(name='movie_meta', con=engine, if_exists='append', index=False)

In [23]:
# Export credits

credits_df.to_sql(name='credits', con=engine, if_exists='append', index=False)

In [24]:
# Export movie ratings

grouped_ratings.to_sql(name='ratings', con=engine, if_exists='append')